# shiny
> components for Shiny

[Shiny for Python](https://shiny.posit.co/py/) is a front end framework that allows you to quickly build simple applications.  It's perfect for customizing your own data annotation and review app for LLMs[^1].  This module contains opinionated components that display [ChatOpenAI](https://api.python.langchain.com/en/latest/chat_models/langchain.chat_models.openai.ChatOpenAI.html) run information in Shiny Apps.

[^1]: We tried other similar frameworks like Gradio, Streamlit, and Panel, but found Shiny to fit our needs the best.

In [ ]:
#|default_exp shiny

In [ ]:
#|export
import os, json
from langfree.transform import RunData
from shiny import module, ui, render, reactive
import shiny.experimental as x
import asyncio

In [ ]:
#|hide
from langfree.runs import _temp_env_var

In [ ]:
#|export
def _get_role(m):
    role = m['role'].upper()
    if 'function_call' in m: return f"{role} - Function Call"
    if role == 'FUNCTION': return 'FUNCTION RESULTS'
    else: return role

def _get_content(m):
    if 'function_call' in m:
        func = m['function_call']
        return f"{func['name']}({func['arguments']})"
    else: return m['content']

def render_input_chat(run:RunData, markdown=True):
    "Render the chat history, except for the last output as a group of cards."
    cards = []
    num_inputs = len(run.inputs)
    for i,m in enumerate(run.inputs):
        content = str(_get_content(m))
        cards.append(
            x.ui.card(
                x.ui.card_header(ui.div({"style": "display: flex; justify-content: space-between;"},
                                    ui.span(
                                        {"style": "font-weight: bold;"}, 
                                        _get_role(m),
                                    ),
                                    ui.span(f'({i+1}/{num_inputs})'),
                                )       
                ),
                x.ui.card_body(ui.markdown(content) if markdown else content),
                class_= "card border-dark mb-3"
            )
        )
    return ui.div(*cards)

In [ ]:
#|hide
tmp_env = {'LANGCHAIN_API_KEY': os.environ['LANGCHAIN_API_KEY_PUB'], 
           'LANGSMITH_PROJECT_ID': os.environ['LANGCHAIN_PROJECT_ID_PUB']}

`render_input` will take an instance of `RunData` and render a set of Shiny cards, with each card containing one turn of the chat conversation:

In [ ]:
from langfree.transform import RunData

In [ ]:
with _temp_env_var(tmp_env):  #context manager that has specific environment vars for testing
    _tst_run = RunData.from_run_id('1863d76e-1462-489a-a8a7-e0404239fe47')
    
_rendered_inp = render_input_chat(_tst_run) 

Below, we render the first card in the conversation:

In [ ]:
_rendered_inp.children[0] # the first message in the conversation

<div class="html-fill-item html-fill-container card bslib-card bslib-mb-spacer card border-dark mb-3" data-bslib-card-init="">
  <div class="card-header">
    <div style="display: flex; justify-content: space-between;">
      <span style="font-weight: bold;">SYSTEM</span><span>(1/3)</span>
    </div>
  </div>
  <div class="html-fill-item html-fill-container card-body bslib-gap-spacing" style="margin-top:auto;margin-bottom:auto;flex:1 1 auto;"><p>You are a helpful documentation Q&amp;A assistant, trained to answer questions from LangSmith's documentation. LangChain is a framework for building applications using large language models.
The current time is 2023-09-05 16:49:07.308007.</p>
<p>Relevant documents will be retrieved in the following messages.</p>
</div>
  <script data-bslib-card-init="">window.bslib.Card.initializeAllCards();</script>
</div>

Here is the last card in the conversation:

In [ ]:
_rendered_inp.children[-1] # the last message in the conversation

<div class="html-fill-item html-fill-container card bslib-card bslib-mb-spacer card border-dark mb-3" data-bslib-card-init="">
  <div class="card-header">
    <div style="display: flex; justify-content: space-between;">
      <span style="font-weight: bold;">USER</span><span>(3/3)</span>
    </div>
  </div>
  <div class="html-fill-item html-fill-container card-body bslib-gap-spacing" style="margin-top:auto;margin-bottom:auto;flex:1 1 auto;"><p>How do I move my project between organizations?</p>
</div>
  <script data-bslib-card-init="">window.bslib.Card.initializeAllCards();</script>
</div>

In [ ]:
#|hide
_run = RunData.from_run_id('59080971-8786-4849-be88-898d3ffc2b45')
_rendered_inp = render_input_chat(_run)
assert len(_rendered_inp.children) == len(_run.inputs)

In [ ]:
#|export
def render_funcs(run:RunData, markdown=True):
    "Render functions as a group of cards."
    cards = []
    if run.funcs:
        num_inputs = len(run.funcs)
        for i,m in enumerate(run.funcs):
            nm = m.get('name', '')
            desc = m.get('description', '')
            content = json.dumps(m.get('parameters', ''), indent=4)
            cards.append(
                x.ui.card(
                    x.ui.card_header(ui.div({"style": "display: flex; justify-content: space-between;"},
                                        ui.span(
                                            {"style": "font-weight: bold;"}, 
                                            nm,
                                        ),
                                        ui.span(f'({i+1}/{num_inputs})'),
                                    )       
                    ),
                    x.ui.card_body(
                        ui.strong(f'Description: {desc}'),
                        ui.markdown(content) if markdown else content
                    ),
                    class_= "card border-dark mb-3"
                )
            )
    return ui.div(*cards)

Similar to `render_input`, `render_funcs` will take an instance of `RunData` and render a set of Shiny cards, with each card containing a function definition that was passed to OpenAI via the [Function Calling API](https://platform.openai.com/docs/guides/gpt/function-calling):

In [ ]:
_rendered_func = render_funcs(_run)
assert len(_run.funcs) == len(_rendered_func.children)

This is one of the functions:

In [ ]:
_rendered_func.children[2]

<div class="html-fill-item html-fill-container card bslib-card bslib-mb-spacer card border-dark mb-3" data-bslib-card-init="">
  <div class="card-header">
    <div style="display: flex; justify-content: space-between;">
      <span style="font-weight: bold;">human-chat</span><span>(3/5)</span>
    </div>
  </div>
  <div class="html-fill-item html-fill-container card-body bslib-gap-spacing" style="margin-top:auto;margin-bottom:auto;flex:1 1 auto;">
    <strong>Description: You can ask a human for guidance when you think you got stuck or you are not sure what to do next. You can also ask him to clarify something or let user you dont know the answer to his request. You can also chat and hold a conversation with the human. Respond to their basic questions. You can also ask human generic questions. The input should be a question for the human.</strong><p>{
&quot;type&quot;: &quot;object&quot;,
&quot;$schema&quot;: &quot;<a href="http://json-schema.org/draft-07/schema#">http://json-schema.org/draft-07/schema#</a>&quot;,
&quot;properties&quot;: {
&quot;input&quot;: {
&quot;type&quot;: &quot;string&quot;
}
},
&quot;additionalProperties&quot;: false
}</p>

  </div>
  <script data-bslib-card-init="">window.bslib.Card.initializeAllCards();</script>
</div>

Here is another function:

In [ ]:
_rendered_func.children[3]

<div class="html-fill-item html-fill-container card bslib-card bslib-mb-spacer card border-dark mb-3" data-bslib-card-init="">
  <div class="card-header">
    <div style="display: flex; justify-content: space-between;">
      <span style="font-weight: bold;">calculator</span><span>(4/5)</span>
    </div>
  </div>
  <div class="html-fill-item html-fill-container card-body bslib-gap-spacing" style="margin-top:auto;margin-bottom:auto;flex:1 1 auto;">
    <strong>Description: Useful for getting the result of a math expression. The input to this tool should be a valid mathematical expression that could be executed by a simple calculator.</strong><p>{
&quot;type&quot;: &quot;object&quot;,
&quot;$schema&quot;: &quot;<a href="http://json-schema.org/draft-07/schema#">http://json-schema.org/draft-07/schema#</a>&quot;,
&quot;properties&quot;: {
&quot;input&quot;: {
&quot;type&quot;: &quot;string&quot;
}
},
&quot;additionalProperties&quot;: false
}</p>

  </div>
  <script data-bslib-card-init="">window.bslib.Card.initializeAllCards();</script>
</div>

In [ ]:
#|export
def render_llm_output(run, width="100%", height="250px"):
    "Render the LLM output as an editable text box."
    o = run.output
    return ui.input_text_area('llm_output', label=ui.h3('LLM Output (Editable)'), 
                              value=o['content'], width=width, height=height)

Below is a demonstration of using `render_llm_output` to produce an editable text box component.  The goal is to allow the user to edit the output for fine tuning to correct errors.

In [ ]:
render_llm_output(_tst_run)

<div class="form-group shiny-input-container" style="width:100%;">
  <label class="control-label" id="llm_output-label" for="llm_output">    <h3>LLM Output (Editable)</h3></label><textarea id="llm_output" class="form-control" style="height:250px;">Currently, LangSmith does not support project migration between organizations. However, you can manually imitate this process by reading and writing runs and datasets using the SDK. Here's an example of exporting runs:

1. Read the runs from the source organization using the SDK.
2. Write the runs to the destination organization using the SDK.

By following this process, you can transfer your runs from one organization to another. However, it may be faster to create a new project within your destination organization and start fresh.

If you have any further questions or need assistance, please reach out to us at support@langchain.dev.</textarea>
</div>

In [ ]:
#|hide
_rendered_out = render_llm_output(_run)
assert _rendered_out.children[1].children[0] == _run.output['content']

In [ ]:
#|export
def invoke_later(delaySecs:int, callback:callable):
    "Execute code in a shiny app with a time delay of `delaySecs` asynchronously."
    async def delay_task():
        await asyncio.sleep(delaySecs)
        async with reactive.lock():
            callback()
            await reactive.flush()
    asyncio.create_task(delay_task())